<a href="https://colab.research.google.com/github/junruren/6.7960-2024-Fall/blob/main/6_7960_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Llama model is access-controlled. Speak with Junru for access.

In [ ]:
!pip install -q -U bitsandbytes accelerate transformers
!pip install boto3
!pip install datasets
!pip install -q rouge-score # For ROUGE
!pip install -q nltk # For BLEU
!pip install -q bert-score # For BERTScore

In [ ]:
from google.colab import userdata
import boto3
from botocore.exceptions import NoCredentialsError, ClientError
import pandas as pd
from datasets import load_dataset
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch
import torch.nn as nn
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Optionally for BERTScore:
from bert_score import score as bert_score

In [ ]:
# Check if CUDA is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

# 8-bit quantization config
quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# Load LLaMA-2-7B
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load the LLM with 8-bit quantization
model_llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16,
).eval()  # Keep LLM in eval mode

# Infer model hidden_size and vocab_size from Llama model config
hidden_size = model_llm.config.hidden_size
vocab_size = model_llm.config.vocab_size

tokenizer = AutoTokenizer.from_pretrained(model_name)

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#-----------------------------------------
# Get Training Data
# https://huggingface.co/datasets/nvidia/OpenMathInstruct-2
# Download only the 1M training split
# Columns: [problem, generated_solution, expected_answer, problem_source]
#-----------------------------------------

dataset = load_dataset('nvidia/OpenMathInstruct-2', split='train_1M', streaming=True) # IterableDataset obj

TRAIN_DATA_SIZE = 70000
VALIDATION_DATA_SIZE = 30000
TEST_DATA_SIZE = 30000

train_data = []
validation_data = []
test_data = []

for example in dataset:
    data = {
        "question": example["problem"],
        "answer": example["expected_answer"]
    }

    if len(train_data) < TRAIN_DATA_SIZE:
        train_data.append(data)
    elif len(validation_data) < VALIDATION_DATA_SIZE:
        validation_data.append(data)
    else:
        test_data.append(data)

    if len(test_data) >= TEST_DATA_SIZE:
        break

print(len(train_data))
print(len(validation_data))
print(len(test_data))

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

[{'question': 'A polynomial of degree two $ax^2 + bx + c$ is such that if you divide the coefficients $a$, $b$, and $c$ by 7, the remainders are 2, 5, and 4, respectively. For what integer value of $k$ does the polynomial have the same remainder when divided by $x + k$ as it does when divided by $x - k$?', 'answer': '0'}, {'question': 'A school is planning a field trip to a museum. If the school can reserve a bus, there is a 90% chance that the trip will take place, but if they cannot reserve a bus, there is only a 30% chance that the trip will occur. According to the bus company, there is a 75% chance that a bus will be available for the school. What is the probability that the field trip will happen? Express your answer as a percent.', 'answer': '75\\%'}, {'question': 'Given that $x + \\frac{1}{x} = \\sqrt{3},$ find $x^{64} + \\frac{1}{x^{64}}.$', 'answer': '-1'}, {'question': 'The average of five consecutive integers is 20. If the middle integer is increased by 5, what is the new av

In [ ]:
#-----------------------------------------
# Construct Initial Prompt
# Define the "prompt for problem-solving prompt" P
#-----------------------------------------

def get_instruction_prompt():
    return "Given the following question, produce a concise and refined prompt to elicit the correct answer to this question from a large language model. The refined prompt should not reveal the solution but should encourage thorough verification and clarity."

def get_context_enhancement_prompt():
    return "You are an intelligent and technologically sophisticated math professor trying to find the right instructions to tell a large language model to explain the correct solution to a math problem."

def get_instruction_enhancement_prompt():
    return "In your prompt, incorporate the exact question and add additional instructions that guide the model to reason, carefully check its reasoning, and provide a correct and well-explained answer."

def get_zero_shot_enhancement_prompt():
    return "Add \"Let's think step by step.\" to the end of your prompt."

def get_question_suffix():
    return " Question: "

def get_prompt_1():
    return get_instruction_prompt() + get_question_suffix()

def get_prompt_2():
    return get_instruction_prompt() + "\n" + get_instruction_enhancement_prompt() + get_question_suffix()

def get_prompt_3():
    return get_context_enhancement_prompt() + "\n" + get_instruction_prompt() + "\n" + get_instruction_enhancement_prompt() + get_question_suffix()

def get_prompt_4():
    return get_context_enhancement_prompt() + "\n" + get_instruction_prompt() + "\n" + get_instruction_enhancement_prompt() + "\n" + get_zero_shot_enhancement_prompt() + get_question_suffix()


In [ ]:
#-----------------------------------------
# The Refiner model:
# We'll define a simple model that takes the LLM hidden states
# and outputs a refined prompt as logits over tokens.
#-----------------------------------------
class SimpleRefiner(nn.Module):
    def __init__(self, hidden_size=4096, vocab_size=32000, seq_len=10):
        super().__init__()
        self.seq_len = seq_len
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size

        # We'll pool over sequence dimension
        self.pool = nn.AdaptiveAvgPool1d(1)

        # MLP from hidden_size to seq_len*vocab_size
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, seq_len * vocab_size)
        )

    def forward(self, llm_hidden_states):
        # llm_hidden_states: [batch, seq_len, hidden_size]
        # Transpose for pooling: [batch, hidden_size, seq_len]
        x = llm_hidden_states.transpose(1, 2)
        # pool -> [batch, hidden_size, 1]
        x = self.pool(x).squeeze(-1) # [batch, hidden_size]

        out = self.mlp(x) # [batch, seq_len*vocab_size]
        out = out.view(-1, self.seq_len, self.vocab_size)
        return out

def text_to_tokens(text):
    return tokenizer.encode(text, return_tensors='pt').to(device)

def tokens_to_text(tokens):
    return tokenizer.decode(tokens, skip_special_tokens=True)

def generate_hidden_states(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids, output_hidden_states=True)
    hidden_states = outputs.hidden_states[-1] # last layer hidden states
    return hidden_states

In [ ]:
def make_final_input_text(refined_prompt_text, question):
    """
    Utility to make the final_input_text that combines the refined prompt and question so that the LLM can generate the answer. Optionally, we may suffix the text with a clear instruction to LLM on how to format the final answer so that our evaluation code can programmatically extract the answer portion.

    Return: (the final_input_text, answer parser function corresponding to the added instruction)
    """
    def answer_parser(output_text: str):
        """
        Extract text immedieately trailing last instance of "Answer: " from output_text.

        Return: the extracted answer, or None if not found.
        """
        answer_lines = [line for line in output_text.split('\n') if line.startswith('Answer: ')]
        if answer_lines:
            answer = answer_lines[-1].replace('Answer: ', '')
            return answer if len(answer) else None
        else:
            return None
    final_input_text = refined_prompt_text.strip() + ": " + question + " Please present your final answer at the end in a new line prefixed with \"Answer: \""
    return final_input_text, answer_parser

class OutputQualityChecker:
    def __init__(self):
        self.total_count = 0
        self.answer_found_count = 0

    def check(self, output_text, answer_parser):
        self.total_count += 1
        if output_text and answer_parser:
            answer = answer_parser(output_text)
            if answer:
                self.answer_found_count += 1

    def report_quality(self):
        print(f"Total count: {self.total_count}")
        print(f"Answer found count: {self.answer_found_count}")
        print(f"Quality: {self.answer_found_count * 100 / self.total_count}% of the responses can parse out the answer as we expected")

In [ ]:
def train_model_with_experiment(name, P, epochs, train_data):
    print(f"P before training: {P}")
    output_quality_checker = OutputQualityChecker()

    for epoch in range(epochs):
        for example in train_data:
            question = example["question"]
            answer = example["answer"]

            # Step 1: Prepare input for LLM: "P + Q"
            input_text = P + question
            input_ids = text_to_tokens(input_text)

            # Sanity check: print LLM's raw completion (initial output)
            with torch.no_grad():
                gen_ids = model_llm.generate(input_ids, max_new_tokens=50)
                gen_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
                # print("LLM initial completion:\n", gen_text, "\n")

            # Step 2: Run LLM to get hidden states
            with torch.no_grad():
                outputs = model_llm(input_ids, output_hidden_states=True)
                llm_hidden_states = outputs.hidden_states[-1]  # [1, seq_len, hidden_size]

            # Step 3: Run refiner
            refiner.train()
            refiner_logits = refiner(llm_hidden_states)  # [1, seq_len=10, vocab_size]

            # Step 4: Convert refiner_logits to tokens (non-differentiable)
            refined_prompt_ids = torch.argmax(refiner_logits, dim=-1)  # [1, 10]
            refined_prompt_text = tokens_to_text(refined_prompt_ids[0])

            # Step 5: Concatenate refined prompt and question
            final_input_text, answer_parser = make_final_input_text(refined_prompt_text, answer)
            final_input_ids = text_to_tokens(final_input_text)

            # Another sanity check: print LLM's output with the refined prompt
            with torch.no_grad():
                gen_refined_ids = model_llm.generate(final_input_ids, max_new_tokens=50)
                gen_refined_text = tokenizer.decode(gen_refined_ids[0], skip_special_tokens=True)
                # print("LLM output after refined prompt:\n", gen_refined_text, "\n")
                output_quality_checker.check(gen_refined_text, answer_parser)

            # Step 6: Compute loss with the known answer
            combined_input = final_input_text + " " + answer
            combined_ids = text_to_tokens(combined_input)

            # Compute LM loss
            outputs = model_llm(combined_ids, labels=combined_ids)
            loss = outputs.loss

            # Step 7: Backprop into refiner
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # print(f"Loss: {loss.item()} | Refined Prompt: {refined_prompt_text}")
            # print("------------------------------------------------------------------------------------")
    print(f"Training done for experiment {experiment}.")
    output_quality_checker.report_quality()

Training done.


In [ ]:
# FOUNDATIONAL EXPERIMENT
# experiment = {
#     "name": "FOUNDATIONAL",
#     "prompt": INITIAL_PROMPT
# }

INITIAL_PROMPT = "Given the following question, produce a concise and refined prompt by incorporating the exact question and adding additional instructions that guide the model to reason step-by-step, carefully check its reasoning, and provide a correct and well-explained answer. The refined prompt should not reveal the solution but should encourage thorough verification and clarity."
epochs = [5, 10, 15, 20]

for epoch in epochs:
    print(f'Running epoch {epoch}...')

    refiner = SimpleRefiner(hidden_size=hidden_size, vocab_size=vocab_size, seq_len=10).to(device).half()
    optimizer = AdamW(refiner.parameters(), lr=1e-4)
    P = experiment["prompt"]

    train_model_with_experiment(experiment["name"], P, EXPERIMENT_EPOCHS, train_data)
    test(test_name, model_llm, tokenizer, trained_refiner, P, validation_data)


P before training: Given the following question, produce a concise and refined prompt by incorporating the exact question and adding additional instructions that guide the model to reason step-by-step, carefully check its reasoning, and provide a correct and well-explained answer. The refined prompt should not reveal the solution but should encourage thorough verification and clarity. Question: 
LLM initial completion:
 Given the following question, produce a concise and refined prompt by incorporating the exact question and adding additional instructions that guide the model to reason step-by-step, carefully check its reasoning, and provide a correct and well-explained answer. The refined prompt should not reveal the solution but should encourage thorough verification and clarity. Question: A polynomial of degree two $ax^2 + bx + c$ is such that if you divide the coefficients $a$, $b$, and $c$ by 7, the remainders are 2, 5, and 4, respectively. For what integer value of $k$ does the p

In [ ]:
# Initialize experiments list
experiments = []

In [ ]:
# ZERO-SHOT PROMPT EXPERIMENTS



In [ ]:
# RUN EXPERIMENTS
"""
Experiment Obj
    name
    prompt
"""

EXPERIMENT_EPOCHS = 1

for experiment in experiments:
    print(f'Running experiment {experiment["name"]}...')

    refiner = SimpleRefiner(hidden_size=hidden_size, vocab_size=vocab_size, seq_len=10).to(device).half()
    optimizer = AdamW(refiner.parameters(), lr=1e-4)
    P = experiment["prompt"]

    train_model_with_experiment(experiment["name"], P, EXPERIMENT_EPOCHS, train_data)
    test(test_name, model_llm, tokenizer, trained_refiner, P, test_data)

    # TODO: write outputs to cloud

# Evaluation of Prompt Refiner Model with Llama2

When evaluating a question-answering (QA) system, we want to measure how closely the model’s generated answers match the ground truth. Here are a few metrics and why they matter:

1. **Exact Match (EM):**  
   *What it measures:*  
   The percentage of answers that match the reference answer exactly, word-for-word.  
   *Significance:*  
   This is a strict metric that gives a clear sense of how often the model gets the answer perfectly right without any extra words or differences in phrasing. It’s easy to understand but can be overly harsh on answers that are correct in meaning but differ slightly in formatting.

2. **F1 Score (Token Overlap):**  
   *What it measures:*  
   The harmonic mean of precision and recall of word overlap between the predicted answer and the reference.  
   *Significance:*  
   F1 captures partial correctness. If the model includes most of the correct information but adds some extra words, it still gets partial credit. This metric balances being lenient and strict, encouraging more complete answers without penalizing minor wording differences as harshly as Exact Match.

3. **ROUGE (Recall-Oriented Understudy for Gisting Evaluation):**  
   *What it measures:*  
   Overlap of n-grams (short sequences of words) between the generated answer and the reference answer.  
   *Significance:*  
   ROUGE was popularized in summarization tasks, but for QA it can reward answers that capture the key content. It’s less strict than EM and can show how similar the answers are at a lexical level.

4. **BLEU (Bilingual Evaluation Understudy):**  
   *What it measures:*  
   Similar to ROUGE, but originally designed for machine translation. It calculates n-gram overlaps from the perspective of precision.  
   *Significance:*  
   BLEU provides another lens on content similarity. While often used for translation, it can provide a secondary check for QA. However, it’s less common for QA than EM/F1.

5. **BERTScore:**  
   *What it measures:*  
   Uses contextual embeddings (from a model like BERT) to measure semantic similarity between predicted and reference answers.  
   *Significance:*  
   BERTScore goes beyond exact word matches and tries to measure if the model’s answer “means” the same thing as the human answer. This helps when the correct answer can be phrased in many valid ways.

For a QA system, the most common metrics in the research literature are typically Exact Match and F1 (token-level), as they are well-established, simple to interpret, and widely accepted. ROUGE and BLEU can provide additional insights, and BERTScore can be helpful if semantic equivalence matters more than exact wording.

In [1]:
def test(test_name, model_llm, tokenizer, trained_refiner, P, test_data):
    """
    The full test pipeline.

    test_data: List of dicts with keys "question" and "answer" e.g. [{"question": "Q?", "answer": "A"}, ...]
    """
    refiner = trained_refiner
    def generate_refined_prompt(question):
        # Given Q, we do same steps as training to get refined prompt
        input_text = P + question
        input_ids = text_to_tokens(input_text)
        with torch.no_grad():
            # Get hidden states
            outputs = model_llm(input_ids, output_hidden_states=True)
            llm_hidden_states = outputs.hidden_states[-1]

        # Pass through refiner
        refiner.eval()
        with torch.no_grad():
            refiner_logits = refiner(llm_hidden_states)  # [1, seq_len, vocab_size]
            refined_prompt_ids = torch.argmax(refiner_logits, dim=-1)  # [1, seq_len]
            refined_prompt_text = tokens_to_text(refined_prompt_ids[0])
        return refined_prompt_text.strip()

    def compute_f1(pred, ref):
        # Simple token-based F1
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        common = set(pred_tokens) & set(ref_tokens)
        # Count how many tokens appear in both
        num_common = sum(min(pred_tokens.count(t), ref_tokens.count(t)) for t in common)
        if len(pred_tokens) == 0 or len(ref_tokens) == 0:
            return 0.0
        precision = num_common / len(pred_tokens)
        recall = num_common / len(ref_tokens)
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)

    # Initialize metrics
    exact_matches = 0
    f1_scores = []
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rougeL_scores = []
    bleu_scores = []
    # For BERTScore (optional):
    bert_references = []
    bert_candidates = []

    for example in test_data:
        question = example["question"]
        true_answer = example["answer"]

        # Generate refined prompt
        refined_prompt = generate_refined_prompt(question)

        # Construct final input
        final_input_text = refined_prompt + ": " + question
        final_input_ids = text_to_tokens(final_input_text)

        # Generate answer from LLM
        with torch.no_grad():
            gen_ids = model_llm.generate(final_input_ids, max_new_tokens=50)
        predicted_answer = tokenizer.decode(gen_ids[0], skip_special_tokens=True)

        # Sometimes the predicted answer might contain the prompt and question again.
        # You may need to strip that out. For simplicity, assume the model's answer
        # follows the prompt and question directly:
        # You might want to split by the original question if necessary.
        # For a naive approach:
        if final_input_text in predicted_answer:
            predicted_answer = predicted_answer.replace(final_input_text, '').strip()

        # Compute Exact Match
        if predicted_answer.strip().lower() == true_answer.strip().lower():
            exact_matches += 1

        # Compute F1
        f1_val = compute_f1(predicted_answer, true_answer)
        f1_scores.append(f1_val)

        # Compute ROUGE
        rouge_result = rouge_scorer_obj.score(true_answer, predicted_answer)
        rouge1_scores.append(rouge_result['rouge1'].fmeasure)
        rougeL_scores.append(rouge_result['rougeL'].fmeasure)

        # Compute BLEU
        # BLEU uses tokenized, reference as a list of lists
        ref_tokens = [true_answer.split()]
        pred_tokens = predicted_answer.split()
        bleu_val = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu_val)

        # For BERTScore:
        bert_references.append(true_answer)
        bert_candidates.append(predicted_answer)

    # Aggregate results
    num_examples = len(test_data)
    exact_match_score = exact_matches / num_examples
    avg_f1 = sum(f1_scores) / num_examples
    avg_rouge1 = sum(rouge1_scores) / num_examples
    avg_rougeL = sum(rougeL_scores) / num_examples
    avg_bleu = sum(bleu_scores) / num_examples

    # For BERTScore:
    P, R, F1 = bert_score(bert_candidates, bert_references, lang='en')
    bert_f1 = torch.mean(F1).item()

    # Print results
    print(f"Evaluation Results: {test_name}")
    print(f"Exact Match: {exact_match_score:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")
    print(f"ROUGE-1: {avg_rouge1:.4f}")
    print(f"ROUGE-L: {avg_rougeL:.4f}")
    print(f"BLEU: {avg_bleu:.4f}")
    print(f"BERTScore-F1: {bert_f1:.4f}")
